<a href="https://colab.research.google.com/github/Rotem2411/alephBERTgimmelDalet/blob/main/Sentiment_LDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import pandas as pd
import torch
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from collections import Counter
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def read_csv_file(file_path):
    data = []
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            data.append(row)
    return data

In [3]:
file_path = 'sentiments.csv'
df = pd.read_csv(file_path)

Take a peek of the data

In [4]:
num_samples = len(df)
print("Total number of samples: ", num_samples)
print(df.head(10))

Total number of samples:  75151
   id                                               text tag category class  \
0   1                    האריות של הצל חזק פה בתגובות...   ש  ECONOMY    b1   
1   2  זמרת תעשייה רק מפרסומות מפורסמת\n  אבל אין קהל...   ש  ECONOMY    b1   
2   4  את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...   ש  ECONOMY    b1   
3   5  שמעון\n  משתתף בפרסומת למילקי במקום לעורר מודע...   ש  ECONOMY    b1   
4   6  הצחקתם אותי\n  מה כל כך אמייזינג בחברה עם אפס ...   ש  ECONOMY    b1   
5   7  איסו חברת הייטק מצליחה צריכה פרסומות שיבואו לע...   ש  ECONOMY    b1   
6   8           לא מכיר אותה ולא שמעתי אפילו שיר אחד שלה   ש  ECONOMY    b1   
7   9  וואללה לא יודע מה מתלהבים מנגה ארז. זה בכלל לא...   ש  ECONOMY    b1   
8  10  אהובה שגיא\n  נגה גדולה.זמרת מעולה.פרסומת מדלי...   ח  ECONOMY    b1   
9  11  הפרסומת של אמדוקס נראית מאוד קודרת ואפורה, כמו...   ש  ECONOMY    b1   

   total_tags  selected_tag  polarity  
0           2             2       1.0  
1           2     

In [5]:
# Sentiment distribution
Sentiment_distribution = {}
for index, row in df.iterrows():
    Sentiment_label = row['tag']
    if Sentiment_label not in Sentiment_distribution:
        Sentiment_distribution[Sentiment_label] = 1
    else:
        Sentiment_distribution[Sentiment_label] += 1
print("Sentiment Distribution:")
for Sentiment_label, count in Sentiment_distribution.items():
    print(f"{Sentiment_label}: {count} tags")

Sentiment Distribution:
ש: 60841 tags
ח: 6982 tags
נ: 7328 tags


In [6]:
# Category distribution
Category_distribution = {}
for index, row in df.iterrows():
  Category_label = row['category']
  if Category_label not in Category_distribution:
    Category_distribution[Category_label] = 1
  else:
    Category_distribution[Category_label] += 1
print("Category Distribution:")
for Category_label, count in Category_distribution.items():
  print(f"{Category_label}: {count} category")

Category Distribution:
ECONOMY: 40923 category
NEWS: 24905 category
sport: 9323 category


In [7]:
words = df['text']
labels = df['category']

# **Data Pre-processing**

We will perform the following steps:


*   **Tokenization**: Split the text into sentences and the sentences into words
*   Remove **punctuation**.
*   All **stopwords** are removed.
*   Words are **lemmatized** — words in third person are changed to first person and verbs in past and future tenses are changed into present.
*   Words are **stemmed** — words are reduced to their root form.

In [8]:
def cleanwords(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('hebrew')
    tokens = nltk.word_tokenize(text)
    no_stopwords = [word for word in tokens if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    return lemm_text

Before cleaning the text:

In [9]:
words[0]

'האריות של הצל חזק פה בתגובות...'

After cleaning the text:

In [10]:
cleanwords(words[0])

['האריות', 'הצל', 'חזק', 'בתגובות']

In [11]:
clean_words = {}
for i in words.index:
  clean_words[i] = cleanwords(words[i])

Splitting the data to train(80%), valid(10%) and test(10%)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(clean_words,labels,test_size = 0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

print("Data distribution:\n- Train: {} \n- Validation: {} \n- Test: {}".format(len(y_train),len(y_val),len(y_test)))

Data distribution:
- Train: 60120 
- Validation: 7515 
- Test: 7516


In [13]:
X_train[0]

['לפי', 'שהמורה', 'לגאומטריה', 'לימדה', 'צודקים', 'הפעם']

In [14]:
X_train = [" ".join(x) for x in X_train]

In [16]:
def vectorize(data, tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names_out()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

LDA - Latent Dirichlet allocation

In [24]:
tfidf_vect = TfidfVectorizer()
tfidf_vect_fit=tfidf_vect.fit(X_train)
X_train = vectorize(X_train, tfidf_vect_fit)
tfidf_matrix = tfidf_vect.fit_transform(X_train)
num_topics = 3
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=100)
lda_matrix = lda_model.fit_transform(tfidf_matrix)
# Print the top words for each topic
feature_names = tfidf_vect.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-31:-1]]
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

Topic 1: תטפח, שבשום, מייננת, איימים, במיגזר, שמסננים, דורכת, עירם, יומה, המוסדיים, התגשם, משדה, לתקצב, הכפרה, מהסדר, מוותרת, למחירם, שרשם, מנקי, המצומצם, מיזוגניים, תאתרו, לסגוד, בשגיאות, לתפוזים, מבוגרות, והאחרונה, ריססו, מצבור, המזרן
Topic 2: פסיכיאטרים, שהתחלפה, חנזיר, ממגע, והשגוי, נדל, מסודרות, התספורות, ייגבו, היזמין, להחלים, טכנים, סתוק, חבצע, אוטורו, שמתקדמים, ומסרבלים, החקירות, מחסן, הענף, למוקד, הקניית, ומסרים, מענים, מהמככבים, מהעבודה, בכללם, לעמדתם, לקולות, לייצמן
Topic 3: ניסנובסטוק, פליישר, למשפטים, כככככככןןןןןןןןןןןןןןן, הלהג, בילדים, כהודאה, ביטוחעם, מתחרטת, סרת, האומנם, הזינוק, ממרוקאי, למפזר, שמרנות, מתגעגעים, כבנט, וגולת, שיקרתם, כיסי, קלט, תפילטר, בטובת, רחבה, מרפדיה, זידאן, הריסות, הריקה, כשהציעו, ברור


In [22]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# Generate the word cloud.
text = str(feature_names)
text = cleanwords(text)
wordcloud = WordCloud().generate(text)

# Display the word cloud.
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title(f"Top words for {topic_idx}", fontsize=16)
plt.show()

TypeError: expected string or bytes-like object

In [ ]:
text = str(feature_names)
text

In [ ]:
text = cleanwords(text)
text